In [1]:

def get_sequences(file_name):
    sequences = []
    lines = []
    with open(file_name, "r") as input_file:
        lines = list(filter(None, input_file.read().split("\n")))

    parts = []
    for line in lines:
        if line.startswith(">"):
            if parts:
                sequences.append("".join(parts))
            parts = []
        else:
            parts.append(line)
    if parts:
        sequences.append("".join(parts))
    return sequences


In [10]:
training_sequences = get_sequences("map_training_sequence.txt")
print(training_sequences)

['...']


In [11]:
def process_data(sequences, max_length, min_length,  target):
    input_output_pairs = []
    for seq in sequences:
        if "X" not in seq and "B" not in seq:
            for i in range(min_length,  max_length+1):
                for start in range(len(seq)-i):
                    end = start + i
                    seq_in = seq[start:end]
                    if calculate_mass(seq_in) == target:
                        input_output_pairs.append( seq_in)

    return input_output_pairs

In [18]:
# scaffold = "-MTQSPSSLSASVGDRVTITCK-NIDKYLNWYQQKPGKAPKLLIYNTNNLQTGVPS\
# RF-G-FTFTI-YCLQHISRPRTFGQGTKVEIKRTVAAPSVFIFPP\
# SDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLT\
# LSKADYEKHKVYACEVTHQGLSSPVTKSFN-"
# scaffold = "-LSLSPGERATLSC-SVSSSYLAWYQQKPGQAPRLLIYDASTRATGIPDRF-SGAD\
# FLLTISSLEPEDFAMYYCQQYGRSPYTFGPGTKVDIKRTVAAPSVFIFPP-LKSGTASVVCLLNNFYP\
# REAKVQWKVDNALQSGNSQESVTEQDSKD-LSSTLTLSKADYEKHKV-\
# GLSSPVTKSFN-"
scaffold = "-MTQSPSSLSASVGDRVTITCK-NIDKYLNWYQQKPGKAPKLLIYNTNNLQTGVPS\
RF-G-FTFTI-YCLQHISRPRTFGQGTKVEIKRT-SVFIFPP\
SDEQLKSGTASVVCLLNNFY-QSGNSQESVTEQ-TYSLSSTLT\
L-YACEVTHQGLSSPVTKSFN-"
contigs = {}
count = 0
for i, j in enumerate(scaffold):
    right_contig = ""
    left_contig = ""
    if j == "-":
        if i == 0:
            for k in range(1,5):
                if scaffold[i+k] == "-":
                    break
                else:
                    right_contig  += scaffold[k]
                    left_contig = ""
            contigs[i] = {"left_contig":left_contig,"right_contig":right_contig}
        elif i== len(scaffold)-1:
            for k in range(4,0,-1):
                if scaffold[i-k] == "-":
                    break
                else:
                    left_contig  += scaffold[i-k]
                    right_contig = ""
            contigs[i] = {"left_contig":left_contig,"right_contig":right_contig}
        else:
            for k in range(1,5):
                if scaffold[i+k] == "-":
                    break
                else:
                    right_contig  += scaffold[i+k]
            
            for k in range(1,5):
                if scaffold[i-k] == "-":
                    break
                else:
                    left_contig  += scaffold[i-k]
            
            contigs[i] = {"left_contig":left_contig[::-1],"right_contig":right_contig}


In [25]:

def count_pep( search_string, side_length):
    def find_and_extract_words(base_string, search_string, side_length):
        results = []
        start_index = 0

        while True:
            index = base_string.find(search_string, start_index)
            

            if index == -1:
                break
            
            start_extract = max(0, index - side_length)
            end_extract = min(len(base_string), index + len(search_string) + side_length)
            
            word = base_string[start_extract:end_extract]
            
            results.append(word)

            start_index = index + 1

        return results

    all_extracted_words = []

    for base_string in training_sequences:
        extracted_words = find_and_extract_words(base_string,search_string, side_length)
        all_extracted_words.extend(extracted_words)

    return all_extracted_words


from itertools import product


# A simplified list of amino acids and their monoisotopic masses (in Daltons)
amino_acids = {
    'G': 57, 'A': 71, 'S': 87, 'P': 97, 'V': 99, 'T': 101,
    'C': 103, 'I': 113, 'L': 113, 'N': 114, 'D': 115, 'Q': 128,
    'K': 128, 'E': 129, 'M': 131, 'H': 137, 'F': 147, 'R': 156,
    'Y': 163, 'W': 186
}

# Function to calculate the mass of a peptide string
def calculate_mass(peptide):
    return sum(amino_acids[aa] for aa in peptide)


Max_seq = {}
Max_p = {}
temp_seq = {}
gap_contig = {}
data = [356, 286 , 231, 360,1204, 338 ,1634,532, 1185,445]
# data = [1025, 442, 288 , 459 , 438, 931, 445]
c =0

for j, k in contigs.items():
    target_mass = data[c]
    tolerance = 0  

    # Find the smallest and largest amino acid masses
    smallest_aa_mass = min(amino_acids.values())
    largest_aa_mass = max(amino_acids.values())
    min_length = max(1, target_mass // largest_aa_mass)
    max_length = target_mass // smallest_aa_mass
    
    valid_peptides = set(process_data(training_sequences, max_length, min_length, target_mass))
    for peptide in valid_peptides:
        total_pep = count_pep( peptide, 5)
        l_c = contigs[j]["left_contig"]
        r_c = contigs[j]["right_contig"]
        target = l_c+ peptide + r_c
        count1 = sum(sequence.count(target) for sequence in total_pep)
        total_count = len(total_pep)
        total_pep = [item for item in total_pep if target not in item]
        target =  l_c[len(l_c)-4:] + peptide + r_c[:4]
        total_pep = [item for item in total_pep if target not in item]
        target =  l_c[len(l_c)-3:] + peptide + r_c[:3]
        total_pep = [item for item in total_pep if target not in item]
        target =  l_c[len(l_c)-2:] + peptide + r_c[:2]
        total_pep = [item for item in total_pep if target not in item]
        target =  l_c[len(l_c)-1:] + peptide + r_c[:1]
        count2 = sum(sequence.count(target) for sequence in total_pep) 
        
        Max_seq[peptide] = count1
        temp_seq[peptide] = [count1, count2]

    max_ti = max(temp_seq, key=lambda k: (temp_seq[k][0], -temp_seq[k][1]))
    gap_contig[max_ti] = {"count": temp_seq[max_ti], "index": j}
    # print(Max_seq)
    # print(Max_prob)
    Max_seq = {}
    temp_seq = {}
    c+=1
# Output the results
for gap, info in gap_contig.items():
    print(f"Gap at index {info['index']} best filled by {gap} with max(alpha(t_i)) and min(beta(t_i)) {info['count']}")


Gap at index 0 best filled by DIQ with max(alpha(t_i)) and min(beta(t_i)) [572, 3]
Gap at index 22 best filled by ASQ with max(alpha(t_i)) and min(beta(t_i)) [3, 9]
Gap at index 58 best filled by SGS with max(alpha(t_i)) and min(beta(t_i)) [734, 31]
Gap at index 60 best filled by SGTD with max(alpha(t_i)) and min(beta(t_i)) [70, 24]
Gap at index 66 best filled by SSLQPEDIATY with max(alpha(t_i)) and min(beta(t_i)) [8, 2]
Gap at index 90 best filled by VAAP with max(alpha(t_i)) and min(beta(t_i)) [891, 6]
Gap at index 118 best filled by PREAKVQWKVDNAL with max(alpha(t_i)) and min(beta(t_i)) [972, 3]
Gap at index 131 best filled by DSKDS with max(alpha(t_i)) and min(beta(t_i)) [978, 0]
Gap at index 142 best filled by SKADYEKHKV with max(alpha(t_i)) and min(beta(t_i)) [935, 0]
Gap at index 162 best filled by RGEC with max(alpha(t_i)) and min(beta(t_i)) [886, 0]
